In [ ]:
import pandas as pd
import scipy
from scipy.optimize import curve_fit

In [ ]:

# function for fitting with relation to T_in and T_out
# x=T_in, y=T_out, z= size to fit

def fit_simple(x,y,z):
    p0=[0.1,0.001,1.] # starting values
    a=(x,y,z) 
    para,_ = scipy.optimize.leastsq(func_simple_zero,p0,args=a)
    return para

def func_simple_zero(para, x, y, z):
    k1,k2,k3 = para
    z_calc = k1*x + k2*y + k3
    z_diff = z_calc - z
    return z_diff

# Function to calculate z using parameters and any x and y:
def func_simple(para, x, y):
    k1,k2,k3 = para
    z = k1*x + k2*y + k3
    return z

#Get paramters: 

t~450 sek

In [ ]:
#get normalized Parameters 
data_key = pd.read_csv(r'output/database_keymark_average_normalized_modus_group.csv')
Models=data_key['Model'].values.tolist()
Models = list(dict.fromkeys(Models))#get models

Group=[]
Pel_n=[]
Pth_max=[]
k1=[]
k2=[]
k3=[]
k4=[]
k5=[]
k6=[]
k7=[]
k8=[]
k9=[]

for model in Models:
    data_key = pd.read_csv(r'output/database_keymark_average_normalized_modus_group.csv')
    data_key = data_key.rename(columns={'P_el [W]': 'P_el', 'P_th [W]': 'P_th', 'T_in [°C]': 'T_in', 'T_out [°C]': 'T_out'})
    data_key = data_key.loc[data_key['Model'] == model]#get data of model
    group = data_key.Group.array[0]#get Group of model
    Pel_ref=data_key.loc[data_key['P_el_n']==1,['P_el']].values.tolist()[0][0]
    Pth_52=data_key.loc[data_key['T_out']==52,['P_th']].values.tolist()[0][0]
    K = 273.15
    eta_carnot_key = (data_key['T_out']+K) / ((data_key['T_out']+K)-(data_key['T_in']+K))
    data_key['eta'] = data_key['COP'] / eta_carnot_key
    data_key.fillna(0, inplace=True)
    variables=['P_el_n', 'P_th_n', 'COP', 'eta']
        
    for var in variables: #get all parameters
        vars()[var+'_para_key'] = fit_simple(data_key['T_in'],data_key['T_out'],data_key[var])
        data_key[var+'_fit'] = func_simple(globals()[var+'_para_key'], data_key['T_in'], data_key['T_out'])
        data_key[var+'_fit_err'] = (data_key[var+'_fit'] - data_key[var]) / data_key[var] * 100
        d = data_key[var+'_fit_err'].mean(), data_key[var+'_fit_err'].max(), data_key[var+'_fit_err'].min()
        vars()[var+'_err'] = pd.DataFrame(d, index=['mean', 'max', 'min'])
    #write Parameters in List
    k1.append(P_th_n_para_key[0])
    k2.append(P_th_n_para_key[1])
    k3.append(P_th_n_para_key[2])
    k4.append(P_el_n_para_key[0])
    k5.append(P_el_n_para_key[1])
    k6.append(P_el_n_para_key[2])
    k7.append(COP_para_key[0])
    k8.append(COP_para_key[1])
    k9.append(COP_para_key[2])
    Group.append(group)
    Pel_n.append(Pel_ref)
    Pth_max.append(Pth_52)
#write List  in Dataframe

paradf=pd.DataFrame()
paradf['Model']=Models
paradf['k1']=k1 
paradf['k2']=k2
paradf['k3']=k3
paradf['k4']=k4
paradf['k5']=k5
paradf['k6']=k6
paradf['k7']=k7
paradf['k8']=k8
paradf['k9']=k9
paradf['Group']=Group
paradf['P_el_n']=Pel_n
paradf['P_th_max']=Pth_max

para = paradf
key = pd.read_csv(r'output/database_keymark_average_normalized_modus_group.csv', delimiter=',')
key=key.loc[key['T_out [°C]']==52]
parakey=para.merge(key, how='left', on='Model')
parakey = parakey.rename(columns={'Group_x': 'Group','P_el_n_x': 'P_el_n [W]','Prated [W]': 'Prated [kW]','P_th_max':'P_th_max [W]'})
table=parakey[['Manufacturer', 'Model', 'Type', 'Modus','Refrigerant','Mass of Refrigerant [kg]','Date','SPL indoor [dBA]','SPL outdoor [dBA]','Prated [kW]','PSB [W]','Guideline','Climate','P_el_n [W]','P_th_max [W]','k1','k2','k3','k4', 'k5','k6','k7','k8','k9', 'Group']]

filt1 = (table['k4'] > 0) & (table['Group']==2)
table.loc[filt1, 'Group'] = 5
table.loc[filt1, 'Modus'] = 'On-Off'
table.to_csv('hplib.csv', encoding='utf-8', index=False)



add Average Heatpumps:

In [ ]:
data_key = pd.read_csv('hplib.csv', delimiter=',')
Power=[1000,3000,5000,10000]
for levels in Power:
    Groups=[1,2,4,5]
    for group in Groups:
        if group==1:
            Type='Outdoor Air/Water'
            modus='Inverter'
        if group==2:
            Type='Brine/Water'
            modus='Inverter'
        if group==4:
            Type='Outdoor Air/Water'
            modus='On-Off'
        if group==5:
            Type='Brine/Water'
            modus='On-Off'
        Group1=data_key.loc[data_key['Group']==group]
        k1_average=Group1['k1'].mean(0)
        k2_average=Group1['k2'].mean(0)
        k3_average=Group1['k3'].mean(0)
        k4_average=Group1['k4'].mean(0)
        k5_average=Group1['k5'].mean(0)
        k6_average=Group1['k6'].mean(0)
        k7_average=Group1['k7'].mean(0)
        k8_average=Group1['k8'].mean(0)
        k9_average=Group1['k9'].mean(0)
        data_key.loc[len(data_key.index)]=['average', str(levels)+'w',Type,modus,'','','','','',levels/1000,'','','average',levels,levels, k1_average,k2_average,k3_average,k4_average,k5_average,k6_average,k7_average,k8_average,k9_average,group]
#data_key.drop(['Group'], axis=1,inplace=True)
data_key.to_csv('data_key_para.csv', encoding='utf-8', index=False)